In [128]:
import pandas as pd
import numpy as np
import math
from functools import reduce

In [2]:
xls = pd.ExcelFile('Data+campus_challenge_FINAL.xlsx')
dat = pd.read_excel(xls, 'Individuals')
print(dat.shape)
CF = pd.read_excel(xls, 'Carbon Footprint', header=[1])
CF.drop(CF.columns[13:], axis=1, inplace=True) # drop unnamed column and notes 
print(CF.shape)

(27054, 16)
(27, 13)


In [3]:
# Activity groups
groups = dat[dat.Indnum == 1][['Group', 'Activity']]
groups.to_csv('Activity_groups.csv', sep=',', encoding='utf-8')
groups[:2]

,Group,Activity
0,1,Household heating => 70F
1,1,Household heating < 70F


In [4]:
# Align Individual fuel type specifications with CF columns 
dat['waste_mgmt'] = 1.0
original = list(dat.columns)
new = ['Indnum','Group','Activity','Units','Consumption','QLI','WH_solar','WH_gas','WH_peak_e','WH_OP_e','gas','NG','hybrid','peak_e','OP_e','jet_fuel','waste_mgmt']
surveyNames = dict(zip(original, new))
dat.rename(columns=surveyNames, inplace=True)
dat.to_csv('survey.csv', sep=',', encoding='utf-8')
dat[:2]

,Indnum,Group,Activity,Units,Consumption,QLI,WH_solar,WH_gas,WH_peak_e,WH_OP_e,gas,NG,hybrid,peak_e,OP_e,jet_fuel,waste_mgmt
0,1,1,Household heating => 70F,hours,2.0,88.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
1,1,1,Household heating < 70F,hours,10.0,85.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0


In [5]:
# pivot on QLI
QLI = dat.pivot(index='Indnum', columns='Activity', values='QLI')
QLI.to_csv('QLI.csv', sep=',', encoding='utf-8')
print(QLI.shape)

(1002, 27)


In [6]:
# Abreviate Carbon Footprint column names
original = list(CF.columns)
new = ['Activity', 'Per', 'WH_solar', 'WH_gas', 'WH_peak_e', 'WH_OP_e', 'gas', 'NG', 'jet_fuel', 'waste_mgmt', 'hybrid', 'peak_e','OP_e']
utilNames = dict(zip(original, new))
CF.rename(columns=utilNames, inplace=True)

# Make activity the index
CF.index = CF.Activity
# Align activity names for matching
CF.index = ['Household heating => 70F', 'Household heating < 70F', 'Use of heat pump', 'Use of air conditioner', 'shower - short', 'shower - long (> 3 min)', 'bath', 'wash-up', 'use of dishwasher', 'use of clothes washer', 'use of clothes dryer', 'use of cooking range', 'use of  oven', 'use of self-clean feature of electric oven', 'Small kitchen appliance in the home', 'TV/computer use', 'air travel - large plane', 'air travel - small  plane (<50 seats)', 'car trips- self only', 'car trips - driver and self', 'car trips - 2+ people with multiple end points', 'trips using public ground transportation', 'bags of garbage disposed', 'bags of recycling deposited (negative CF)', 'bags of compost deposited (negative CF)', 'hazardous or electric items disposed', 'large items disposed']
CF.drop(['Activity', 'Per'], axis=1, inplace=True)

# Save to .csv with 0's instead of Nans
CF.to_csv('CF.csv', sep=',', encoding='utf-8') # .fillna(0)
CF[:2]

,WH_solar,WH_gas,WH_peak_e,WH_OP_e,gas,NG,jet_fuel,waste_mgmt,hybrid,peak_e,OP_e
Household heating => 70F,NaN,NaN,NaN,NaN,NaN,0.000436,NaN,NaN,NaN,0.000650,0.000542
Household heating < 70F,NaN,NaN,NaN,NaN,NaN,0.000872,NaN,NaN,NaN,0.000923,0.000901


In [7]:
CF

,WH_solar,WH_gas,WH_peak_e,WH_OP_e,gas,NG,jet_fuel,waste_mgmt,hybrid,peak_e,OP_e
Household heating => 70F,NaN,NaN,NaN,NaN,NaN,0.000436,NaN,NaN,NaN,0.000650,0.000542
Household heating < 70F,NaN,NaN,NaN,NaN,NaN,0.000872,NaN,NaN,NaN,0.000923,0.000901
Use of heat pump,NaN,NaN,NaN,NaN,NaN,0.001074,NaN,NaN,NaN,0.001229,0.001188
Use of air conditioner,NaN,NaN,NaN,NaN,NaN,0.000598,NaN,NaN,NaN,0.007980,0.000721
shower - short,0.000012,0.000102,0.000232,0.000199,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shower - long (> 3 min),0.000017,0.000149,0.000354,0.000312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bath,0.000088,0.000254,0.000412,0.000368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wash-up,0.000004,0.000042,0.000067,0.000055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
use of dishwasher,0.000025,0.000165,0.000398,0.000311,NaN,NaN,NaN,NaN,NaN,0.000084,0.000078
use of clothes washer,0.000033,0.000199,0.000433,0.000382,NaN,0.000154,NaN,NaN,NaN,0.000102,0.000093


In [8]:
# Individual consumption by activity
Cmp = dat.pivot(index='Indnum', columns='Activity', values='Consumption').fillna(0.0)
Cmp.iloc[:,6:8] = Cmp.iloc[:,6:8] / 100 # Set air travel units to 100 miles -- to align with CF units

Cmp.to_csv('Cmp.csv', sep=',', encoding='utf-8')

print(Cmp.shape)
Cmp[:2]

(1002, 27)


Activity,Household heating < 70F,Household heating => 70F,Small kitchen appliance in the home,TV/computer use,Use of air conditioner,Use of heat pump,air travel - large plane,air travel - small plane (<50 seats),bags of compost deposited (negative CF),bags of garbage disposed,...,shower - long (> 3 min),shower - short,trips using public ground transportation,use of oven,use of clothes dryer,use of clothes washer,use of cooking range,use of dishwasher,use of self-clean feature of electric oven,wash-up
Indnum,,,,,,,,,,,,,,,,,,,,,
1,10.0,2.0,0.0,64.0,20.0,0.0,27.47,0.0,3.0,26.0,...,30.0,5.0,15.0,9.0,3.0,2.0,0.0,0.0,3.0,44.0
2,2.0,7.0,0.0,63.0,15.0,0.0,25.24,0.0,0.0,17.0,...,13.0,7.0,12.0,8.0,4.0,8.0,0.0,0.0,3.0,44.0


## Individual Carbon Footprints
Now I will use the Carbon Footprint table along with each individuals specific sources of energy to create a table with individuals unit carbon footprint for each activity. I can then multiply thise by the individual consumption table (Cmp) to get the carbon footprint of each individual -- broken down by activity.

I will then chop this table up into the activity groups, so that I can more easily locate areas with high carbon footprints -- which I will view as areas with the most room for improvement.

In [9]:
# Get relevant sources for each activity
all_sources = [list(CF.loc[i].dropna().index) for i in Cmp.columns]

# dataframes of individual sources used for the relevant sources of each activity
currentSources = [dat.loc[dat.Activity == activity][source].fillna(0) for activity, source in 
                  zip(Cmp.columns, all_sources)]

# get number of different relevant sources used
numSources = [source.sum(axis=1) for source in currentSources]

In [10]:
# Get the activities in each group
group_activities = [list(groups.Activity[groups.Group == ag]) for ag in range(1,7)]

# Get all sources used in the activities of each activity group
group_sources = [list(set([item for sublist in [list(CF.loc[i].dropna().index) for i in g] for item in sublist])) 
                 for g in group_activities]

In [11]:
# all_sources (27) ~ group_sources (6)
# currentSources (27) ~ currentSources_g (6)
# dataframes of individual sources used for the relevant sources of each activity group
currentSources_g = [reduce(lambda x, y: x.add(y, fill_value=0), 
                           [dat.loc[dat.Activity == activity][group_sources[0]].fillna(0).reset_index(drop=True)
                            for activity in gActs]) for gActs in group_activities]
# TODO: numSources_g then run similar algorithm as below to get Individual CF by activity group
# ** Or just reduce CF_unit into the 6 groups 

In [12]:
# This is the CF per unit for each individual using each activity
CF_unit = pd.DataFrame(index=Cmp.index, columns=Cmp.columns)

# Loop through all activities
for i in range(len(Cmp.columns)):
    if min(numSources[i]) == 0:
        # Impute missing sources with CF of highest source or average of peak/ OP electric
        if "peak_e" in currentSources[i].columns:
            sources = ['peak_e', 'OP_e']
        elif "WH_peak_e" in currentSources[i].columns:
            sources = ['WH_peak_e', 'WH_OP_e']
        elif "jet_fuel" in currentSources[i].columns:
            sources = ['jet_fuel']
        elif "gas" in currentSources[i].columns:
            sources = ['gas']
        for idx,_ in currentSources[i].iterrows():
            if sum(_) == 0:
                currentSources[i].loc[idx].loc[sources] = 1.0
        # Update numSources with imputed sources used
        numSources[i] = currentSources[i].sum(axis=1)
    # multiply individual sources used (boolean) by CF for relevant sources
    CF_u = np.matrix(currentSources[i]) * np.matrix(CF[all_sources[i]].loc[Cmp.columns[i]]).T
    # average the CF for each source (if multiple)
    CF_unit[Cmp.columns[i]] = CF_u / np.matrix(numSources[i]).T

In [14]:
CF_unit.to_csv('CF_unit.csv', sep=',', encoding='utf-8')
CF_unit[:2]

Activity,Household heating < 70F,Household heating => 70F,Small kitchen appliance in the home,TV/computer use,Use of air conditioner,Use of heat pump,air travel - large plane,air travel - small plane (<50 seats),bags of compost deposited (negative CF),bags of garbage disposed,...,shower - long (> 3 min),shower - short,trips using public ground transportation,use of oven,use of clothes dryer,use of clothes washer,use of cooking range,use of dishwasher,use of self-clean feature of electric oven,wash-up
Indnum,,,,,,,,,,,,,,,,,,,,,
1,0.000872,0.000436,0.000048,0.000002,0.00435,0.001208,0.0179,0.0408,-0.0419,0.0419,...,0.000333,0.000216,0.000276,0.000156,0.000127,0.000097,0.000139,0.000081,0.000189,0.000042
2,0.000872,0.000436,0.000048,0.000002,0.00435,0.001208,0.0179,0.0408,-0.0419,0.0419,...,0.000333,0.000216,0.000276,0.000150,0.000122,0.000097,0.000135,0.000081,0.000189,0.000004


In [20]:
# Found sources for activities that are not identified in CF

In [24]:
dat[dat.Indnum == 9]

,Indnum,Group,Activity,Units,Consumption,QLI,WH_solar,WH_gas,WH_peak_e,WH_OP_e,gas,NG,hybrid,peak_e,OP_e,jet_fuel,waste_mgmt
216,9,1,Household heating => 70F,hours,4.0,61.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
217,9,1,Household heating < 70F,hours,6.0,89.0,1.0,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN,1.0
218,9,1,Use of heat pump,hours,NaN,55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
219,9,1,Use of air conditioner,hours,20.0,50.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
220,9,2,shower - short,count,2.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
221,9,2,shower - long (> 3 min),count,35.0,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
222,9,2,bath,count,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
223,9,2,wash-up,count,52.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
224,9,3,use of dishwasher,count,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
225,9,3,use of clothes washer,count,7.0,34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [91]:
dat[dat.WH_solar == 1].groupby('Activity')['Consumption'].count()

Activity
Household heating < 70F                       57
Household heating => 70F                      49
Small kitchen appliance in the home            4
TV/computer use                               53
Use of air conditioner                        59
Use of heat pump                               5
bath                                           3
shower - long (> 3 min)                       75
shower - short                                51
use of  oven                                  49
use of clothes dryer                          45
use of clothes washer                         56
use of cooking range                           3
use of dishwasher                              4
use of self-clean feature of electric oven    53
wash-up                                       69
Name: Consumption, dtype: int64

In [34]:
# Loop through CF.columns
col = CF.columns[0]
col

'WH_solar'

In [41]:
# Activities that use CF.column as a source
a = dat[dat[col] == 1].groupby('Activity').count().index
a

Index(['Household heating < 70F', 'Household heating => 70F',
       'Small kitchen appliance in the home', 'TV/computer use',
       'Use of air conditioner', 'Use of heat pump', 'bath',
       'shower - long (> 3 min)', 'shower - short', 'use of  oven',
       'use of clothes dryer', 'use of clothes washer', 'use of cooking range',
       'use of dishwasher', 'use of self-clean feature of electric oven',
       'wash-up'],
      dtype='object', name='Activity')

In [ ]:
# Loop through activites

In [83]:
[(act,CF2.loc[act]['WH_solar']) for act in a]

[('Household heating < 70F', 0.00018685714285714288),
 ('Household heating => 70F', 9.342857142857144e-05),
 ('Small kitchen appliance in the home', nan),
 ('TV/computer use', nan),
 ('Use of air conditioner', 0.00012814285714285716),
 ('Use of heat pump', 0.00023014285714285717),
 ('bath', 8.8e-05),
 ('shower - long (> 3 min)', 1.7e-05),
 ('shower - short', 1.2e-05),
 ('use of  oven', 5.464285714285715e-05),
 ('use of clothes dryer', 4.007142857142857e-05),
 ('use of clothes washer', 3.3e-05),
 ('use of cooking range', 4.307142857142858e-05),
 ('use of dishwasher', 2.5e-05),
 ('use of self-clean feature of electric oven', nan),
 ('wash-up', 4e-06)]

In [76]:
# CF source ratios

In [88]:
CF2

,WH_solar,WH_gas,WH_peak_e,WH_OP_e,gas,NG,jet_fuel,waste_mgmt,hybrid,peak_e,OP_e
Household heating => 70F,0.000093,NaN,NaN,NaN,NaN,0.000436,NaN,NaN,NaN,0.000650,0.000542
Household heating < 70F,0.000187,NaN,NaN,NaN,NaN,0.000872,NaN,NaN,NaN,0.000923,0.000901
Use of heat pump,0.000230,NaN,NaN,NaN,NaN,0.001074,NaN,NaN,NaN,0.001229,0.001188
Use of air conditioner,0.000128,NaN,NaN,NaN,NaN,0.000598,NaN,NaN,NaN,0.007980,0.000721
shower - short,0.000012,0.000102,0.000232,0.000199,NaN,NaN,NaN,NaN,NaN,NaN,NaN
shower - long (> 3 min),0.000017,0.000149,0.000354,0.000312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bath,0.000088,0.000254,0.000412,0.000368,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wash-up,0.000004,0.000042,0.000067,0.000055,NaN,NaN,NaN,NaN,NaN,NaN,NaN
use of dishwasher,0.000025,0.000165,0.000398,0.000311,NaN,NaN,NaN,NaN,NaN,0.000084,0.000078
use of clothes washer,0.000033,0.000199,0.000433,0.000382,NaN,0.000154,NaN,NaN,NaN,0.000102,0.000093


In [74]:
CF2 = CF.copy()
# Use 'use of clothes washer' to relate NG to WH_solar
CF2.WH_solar = [activity.NG * (CF.WH_solar / CF.NG)['use of clothes washer'] if np.isnan(activity.WH_solar) 
                else activity.WH_solar for _,activity in CF2.iterrows()]

## Top 5

In [127]:
IndCF.sum().sort_values(ascending=False)[:15]

Activity
bags of garbage disposed                          940.361700
air travel - large plane                          538.527049
car trips- self only                              197.631578
Use of air conditioner                             96.856519
large items disposed                               21.311000
car trips - driver and self                        14.033928
air travel - small  plane (<50 seats)              11.156760
shower - long (> 3 min)                             7.598916
Household heating < 70F                             4.820524
trips using public ground transportation            4.255644
car trips - 2+ people with multiple end points      3.175026
Household heating => 70F                            2.471765
wash-up                                             2.444820
hazardous or electric items disposed                1.582500
use of  oven                                        1.543032
dtype: float64

In [110]:
# Individual CF for each activity
IndCF = Cmp * CF_unit
a5 = list(IndCF.sum().sort_values(ascending=False)[:5].index)
a5

['bags of garbage disposed',
 'air travel - large plane',
 'car trips- self only',
 'Use of air conditioner',
 'large items disposed']

In [113]:
QLI[a5].loc[1]

Activity
bags of garbage disposed    41.0
air travel - large plane    42.0
car trips- self only        44.0
Use of air conditioner      45.0
large items disposed        15.0
Name: 1, dtype: float64

In [114]:
Cmp[a5].loc[1]

Activity
bags of garbage disposed     26.00
air travel - large plane     27.47
car trips- self only        443.00
Use of air conditioner       20.00
large items disposed          2.00
Name: 1, dtype: float64

In [119]:
IndCF[a5].loc[2]

Activity
bags of garbage disposed    0.712300
air travel - large plane    0.451796
car trips- self only        0.150423
Use of air conditioner      0.065257
large items disposed        0.000000
Name: 2, dtype: float64

In [116]:
from scipy import stats

In [126]:
stats.percentileofscore(IndCF[a5[0]],IndCF[a5[0]].loc[3])

48.35329341317365

In [121]:
IndCF[a5[0]].loc[2]

0.7123

In [122]:
IndCF[a5[0]]

Indnum
1       1.0894
2       0.7123
3       0.9218
4       1.1313
5       0.7123
6       0.9218
7       0.7542
8       0.8380
9       0.9637
10      1.1732
11      1.0475
12      0.8799
13      0.7123
14      0.9637
15      1.0894
16      0.8799
17      0.9218
18      0.9218
19      0.9637
20      1.1732
21      0.8380
22      0.7542
23      0.8799
24      0.9637
25      0.9218
26      0.7123
27      0.7123
28      1.0475
29      0.7961
30      0.7542
         ...  
973     0.9218
974     1.0475
975     0.8380
976     1.0894
977     0.7123
978     0.7123
979     0.8799
980     0.9218
981     1.0475
982     0.7123
983     0.9637
984     0.8799
985     0.9218
986     1.0056
987     1.0475
988     0.8380
989     0.8380
990     0.8380
991     1.0475
992     0.8799
993     0.8380
994     0.8380
995     0.9637
996     0.9218
997     0.7961
998     0.9218
999     1.0475
1000    1.0894
1001    0.7123
1002    1.0894
Name: bags of garbage disposed, Length: 1002, dtype: float64